In [109]:
import json
import pandas as pd
import numpy as np

import PyPDF2
import re

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [110]:
path_Text = "F:/Environmental Baseline Data/Version 2/Data/HTML Images and Tables/A0H8C0 %2D 13.0 EIA %2D Section 13.1 to 13.6/table-of-contents.txt"
path_pdf = "F:/Environmental Baseline Data/Version 2/Data/PDFs/A0H8C0 %2D 13.0 EIA %2D Section 13.1 to 13.6 .pdf"

# Extract Chapter Headings from the TOC text file

### Reading the JSON File as text and then extracting teh main chapter names

Ref -> https://stackabuse.com/reading-and-writing-json-to-a-file-in-python/

In [113]:
def head_clean(head):
    index = 0
    for i in range(len(head)):
        if (head[i] == '.') and (head[i+1] == '.') and (head[i+2] =='.'):
            index = i
            break
        
    head_clean = head[:i]
    return head_clean

In [116]:
with open(path_Text) as json_file:
    data = json.load(json_file)
    raw_TOC_text = data['raw_text'] #currently the array has a lot of strings which are not main headers 
        
    chapter_keys = ["0 Before First Chapter"] 
    for head in raw_TOC_text:
        head = head.strip()
        if head[0].isdigit(): # assuming all the headings in the TOC are numbered
            head = head_clean(head) # removing soo many filler '.' in the string
            
            ##########################################
            # Here, we night need to set a threshold (On the number of dots) value depending on the PDF and its TOC 
            ##########################################
            
            threshold = 2
            if head.count('.') < threshold:
                head = head.strip()
                chapter_keys.append(head)
                
            # Here we check for Appendix 
        if "Appendix" in head:
            chapter_keys.append("99 Appendix")
            break
                
chapter_keys

['0 Before First Chapter',
 '13  Environmental Impact Assessment and Mitigation Plan',
 '13.1 Introduction',
 '13.2 Project Description',
 '13.3 Assessment Methods',
 '13.4 Air',
 '13.5 Terrain and Soils',
 '13.6 Vegetation',
 '13.7  Wildlife and Wildlife Habitat',
 '13.8  Fisheries and Aquatic Resources',
 '13.9  Land and Resource Use',
 '13.10  Traditional Land Use Study',
 '13.11 Heritage/Historical Resources',
 '13.12  Assessment of Upset Events',
 '13.13 References',
 '99 Appendix']

In [117]:
PDF_locations = []
PDF_names = []
Chapter_numbers = []
Chapter_texts = []

for chapter_key in chapter_keys:
    PDF_locations.append(path_pdf)
    PDF_names.append(path_pdf.split('/')[-1])
    index = chapter_key.index(' ')
    Chapter_numbers.append(chapter_key[:index])
    Chapter_texts.append(chapter_key[index+1:])

In [118]:
ChaptersDF = pd.DataFrame({'PDF_names' : PDF_names, 
                           'Chapter_numbers' : Chapter_numbers, 
                           'Chapter_texts' : Chapter_texts, 
                           'Chapter_keys' : chapter_keys, 
                           'PDF_locations' : PDF_locations}, dtype = str)
ChaptersDF

,PDF_names,Chapter_numbers,Chapter_texts,Chapter_keys,PDF_locations
0,A0H8C0 %2D 13.0 EIA %2D Section 13.1 to 13.6 .pdf,0,Before First Chapter,0 Before First Chapter,F:/Environmental Baseline Data/Version 2/Data/...
1,A0H8C0 %2D 13.0 EIA %2D Section 13.1 to 13.6 .pdf,13,Environmental Impact Assessment and Mitigatio...,13 Environmental Impact Assessment and Mitiga...,F:/Environmental Baseline Data/Version 2/Data/...
2,A0H8C0 %2D 13.0 EIA %2D Section 13.1 to 13.6 .pdf,13.1,Introduction,13.1 Introduction,F:/Environmental Baseline Data/Version 2/Data/...
3,A0H8C0 %2D 13.0 EIA %2D Section 13.1 to 13.6 .pdf,13.2,Project Description,13.2 Project Description,F:/Environmental Baseline Data/Version 2/Data/...
4,A0H8C0 %2D 13.0 EIA %2D Section 13.1 to 13.6 .pdf,13.3,Assessment Methods,13.3 Assessment Methods,F:/Environmental Baseline Data/Version 2/Data/...
5,A0H8C0 %2D 13.0 EIA %2D Section 13.1 to 13.6 .pdf,13.4,Air,13.4 Air,F:/Environmental Baseline Data/Version 2/Data/...
6,A0H8C0 %2D 13.0 EIA %2D Section 13.1 to 13.6 .pdf,13.5,Terrain and Soils,13.5 Terrain and Soils,F:/Environmental Baseline Data/Version 2/Data/...
7,A0H8C0 %2D 13.0 EIA %2D Section 13.1 to 13.6 .pdf,13.6,Vegetation,13.6 Vegetation,F:/Environmental Baseline Data/Version 2/Data/...
8,A0H8C0 %2D 13.0 EIA %2D Section 13.1 to 13.6 .pdf,13.7,Wildlife and Wildlife Habitat,13.7 Wildlife and Wildlife Habitat,F:/Environmental Baseline Data/Version 2/Data/...
9,A0H8C0 %2D 13.0 EIA %2D Section 13.1 to 13.6 .pdf,13.8,Fisheries and Aquatic Resources,13.8 Fisheries and Aquatic Resources,F:/Environmental Baseline Data/Version 2/Data/...


# Reading PDF File to find the Chapter Headers

Ref -> https://stackoverflow.com/questions/17098675/searching-text-in-a-pdf-using-python

In [119]:
pdf_paths = [path_pdf]
for pdf_path in pdf_paths:
    # open the pdf file
    object = PyPDF2.PdfFileReader(path_pdf)
    
    # get number of pages
    NumPages = object.getNumPages()
    
    ############################################
    # Here we should call the functions to extract the Chapter Names from the PDFs
    # input: Path of the text file 
    # output: ChaptersDF, chapter_keys
    ############################################
    
        
    # chapter_key_index is set as 1 because we dont want to search for"0 Before First Chapter"
    # the array chapter_key_pages has first element 0 for the same reason
    chapter_key_index = 1
    chapter_key_pages = [0]
    # extract text and do the search
    for i in range(0, NumPages):
        PageObj = object.getPage(i)
        #print("this is page " + str(i)) 
        Text = PageObj.extractText() 
        # print(Text)
        ResSearch = re.search(String, Text)
        print(ResSearch)
    
    


IndentationError: expected an indented block (<ipython-input-119-ce2f9f3c5280>, line 21)

# Rough Work 

In [53]:
len("Figure 13.4-4  Mean Monthly Precipitation and Number of Days with Measurable\n")
len("Figure 13.7-2  Woodland Caribou Habitat Availability during the Early Winter Season..... 13-113\n")
len("Figure 13D.A-8  Typical Existing Bridge ...........................................................................................9\n",)

77

96

133

In [96]:
chapter = chapter_keys[2]
index = chapter.index(".")
index

2

In [67]:
d = {'col1': [1, 2], 'col2': [3, 4]}
df = pd.DataFrame(data=d)
df

,col1,col2
0,1,3
1,2,4


In [69]:
df2 = pd.DataFrame(np.array([[1, 2, 3], [4, 5, 6], [7, 8, 9]]),
                    columns=['a', 'b', 'c'])
df2

,a,b,c
0,1,2,3
1,4,5,6
2,7,8,9
